In [1]:
import os

import mindspore
from mindnlp.core import ops
from mindnlp.transformers import AutoModelForSeq2SeqLM
from mindnlp.peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from mindnlp.dataset import load_dataset

from mindnlp.transformers import AutoTokenizer
from mindnlp.transformers.optimization import get_linear_schedule_with_warmup
from tqdm import tqdm

model_name_or_path = "t5-large"
tokenizer_name_or_path = "t5-large"
checkpoint_name = "financial_sentiment_analysis_prefix_tuning_v1.ckpt"
max_length = 128
lr = 1e-2
num_epochs = 5
batch_size = 8

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
Building prefix dict from the defa

In [2]:
# creating model
peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
trainable params: 983,040 || all params: 738,651,136 || trainable%: 0.13308583065659835


In [3]:
mindspore.dataset.config.set_seed(123)
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/datasets/load.py:1486: FutureWarning: The repository for financial_phrasebank contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/financial_phrasebank
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [4]:
classes = dataset.source.ds.features["label"].names
classes

['negative', 'neutral', 'positive']

In [5]:
train_dataset, validation_dataset = dataset.split([0.9, 0.1])

In [6]:
def add_text_label(sentence, label):
    return sentence, label, classes[label.item()]

train_dataset = train_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])
validation_dataset = validation_dataset.map(add_text_label, ['sentence', 'label'], ['sentence', 'label', 'text_label'])

In [7]:
next(train_dataset.create_dict_iterator())

{'sentence': Tensor(shape=[], dtype=String, value= 'The negotiations concern personnel of Cencorp Corporation and Singulase Oy as whole in Finland and in Sweden , the company said .'),
 'label': Tensor(shape=[], dtype=Int64, value= 1),
 'text_label': Tensor(shape=[], dtype=String, value= 'neutral')}

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

/home/lvyufeng/miniconda3/envs/mindspore/lib/python3.9/site-packages/mindnlp/transformers/tokenization_utils_base.py:1526: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [9]:
import numpy as np
from mindnlp.dataset import BaseMapFunction
from threading import Lock
lock = Lock()

class MapFunc(BaseMapFunction):
    def __call__(self, sentence, label, text_label):
        lock.acquire()
        model_inputs = tokenizer(sentence, max_length=max_length, padding="max_length", truncation=True)
        labels = tokenizer(text_label, max_length=2, padding="max_length", truncation=True)
        lock.release()
        labels = labels['input_ids']
        labels = np.where(np.equal(labels, tokenizer.pad_token_id), -100, labels)
        return model_inputs['input_ids'], model_inputs['attention_mask'], labels


def get_dataset(dataset, tokenizer, shuffle=True):
    input_colums=['sentence', 'label', 'text_label']
    output_columns=['input_ids', 'attention_mask', 'labels']
    dataset = dataset.map(MapFunc(input_colums, output_columns),
                          input_colums, output_columns)
    if shuffle:
        dataset = dataset.shuffle(64)
    dataset = dataset.batch(batch_size)
    return dataset

train_dataset = get_dataset(train_dataset, tokenizer)
eval_dataset = get_dataset(validation_dataset, tokenizer, shuffle=False)

In [10]:
next(train_dataset.create_dict_iterator())

{'input_ids': Tensor(shape=[8, 128], dtype=Int64, value=
 [[   37, 14302,  2410 ...     0,     0,     0],
  [   86,    48,   194 ...     0,     0,     0],
  [ 5433,     3,     6 ...     0,     0,     0],
  ...
  [   37, 12400,  2517 ...     0,     0,     0],
  [   37,   349,   141 ...     0,     0,     0],
  [  328,    56,    36 ...     0,     0,     0]]),
 'attention_mask': Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 'labels': Tensor(shape=[8, 2], dtype=Int64, value=
 [[7163,    1],
  [7163,    1],
  [7163,    1],
  ...
  [7163,    1],
  [7163,    1],
  [7163,    1]])}

In [11]:
# optimizer and lr scheduler
from mindnlp.core.optim import AdamW
optimizer = AdamW(tuple(param for param in model.parameters() if param.requires_grad), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataset) * num_epochs),
)

In [12]:
# training and evaluation
def forward_fn(**batch):
    outputs = model(**batch)
    loss = outputs.loss
    return loss

grad_fn = mindspore.value_and_grad(forward_fn, None, tuple(param for param in model.parameters() if param.requires_grad))

def train_step(**batch):
    loss, grads = grad_fn(**batch)
    optimizer.step(grads)
    return loss

for epoch in range(num_epochs):
    model.set_train()
    total_loss = 0
    train_total_size = train_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(train_dataset.create_dict_iterator(), total=train_total_size)):
        loss = train_step(**batch)
        total_loss += loss.float()
        lr_scheduler.step()

    model.set_train(False)
    eval_loss = 0
    eval_preds = []
    eval_total_size = eval_dataset.get_dataset_size()
    for step, batch in enumerate(tqdm(eval_dataset.create_dict_iterator(), total=eval_total_size)):
        with mindspore._no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.float()
        eval_preds.extend(
            tokenizer.batch_decode(ops.argmax(outputs.logits, -1).asnumpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataset)
    eval_ppl = ops.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataset)
    train_ppl = ops.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 255/255 [02:39<00:00,  1.60it/s]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:07<00:00,  3.91it/s]


epoch=0: train_ppl=Tensor(shape=[], dtype=Float32, value= 5.99932) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 1.79165) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.09334) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.089238)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:07<00:00,  3.77it/s]


epoch=1: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.12824) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.120655) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.05797) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0563556)


...

epoch=2: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.08222) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0790151) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.05312) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0517595)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.51it/s]


epoch=3: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.07526) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0725641) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.05055) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0493116)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 29/29 [00:06<00:00,  4.46it/s]

epoch=4: train_ppl=Tensor(shape=[], dtype=Float32, value= 1.0591) train_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0574166) eval_ppl=Tensor(shape=[], dtype=Float32, value= 1.05313) eval_epoch_loss=Tensor(shape=[], dtype=Float32, value= 0.0517663)


In [13]:
# print accuracy
correct = 0
total = 0

ground_truth = []

for pred, data in zip(eval_preds, validation_dataset.create_dict_iterator(output_numpy=True)):
    true = str(data['text_label'])
    ground_truth.append(true)
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{ground_truth[:10]=}")

accuracy=97.34513274336283 % on the evaluation dataset
eval_preds[:10]=['neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral']
ground_truth[:10]=['neutral', 'neutral', 'positive', 'neutral', 'positive', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral']


In [14]:
next(eval_dataset.create_tuple_iterator())

[Tensor(shape=[8, 128], dtype=Int64, value=
 [[   37,   579,  6126 ...     0,     0,     0],
  [11239, 14434,    16 ...     0,     0,     0],
  [   71,  7410,  3529 ...     0,     0,     0],
  ...
  [   86,  4408,   663 ...     0,     0,     0],
  [  438,   874,   315 ...     0,     0,     0],
  [15715,    19,   788 ...     0,     0,     0]]),
 Tensor(shape=[8, 128], dtype=Int64, value=
 [[1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  ...
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0],
  [1, 1, 1 ... 0, 0, 0]]),
 Tensor(shape=[8, 2], dtype=Int64, value=
 [[7163,    1],
  [7163,    1],
  [1465,    1],
  ...
  [7163,    1],
  [7163,    1],
  [7163,    1]])]

In [15]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [16]:
ckpt = f"{peft_model_id}/adapter_model.ckpt"
!du -h $ckpt

3.8M	t5-large_PREFIX_TUNING_SEQ_2_SEQ_LM/adapter_model.ckpt


In [17]:
from mindnlp.peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [18]:
model.set_train(False)
example = next(validation_dataset.create_dict_iterator(output_numpy=True))

print(example['text_label'])
inputs = tokenizer(example['text_label'], return_tensors="ms")
print(inputs)

with mindspore._no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.asnumpy(), skip_special_tokens=True))

neutral
{'input_ids': Tensor(shape=[1, 2], dtype=Int64, value=
[[7163,    1]]), 'attention_mask': Tensor(shape=[1, 2], dtype=Int64, value=
[[1, 1]])}
[[   0 7163    1]]
['neutral']
